Train disease classifiers with features generated by `reddit_feature_gen.ipynb`.

---

In [66]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import pickle
import ast

In [ ]:
# # included for convenience to help find correct paths
# import os
# os.getcwd()
# os.listdir("..")

Constants.

In [3]:
# paths
MEDRED_REPRODUCIBLE_DIR = "../"
# are features for the DL implementation or MetaMap?
EMBEDDING_TYPE = "DL"
# features and results
FEATURES_IN = MEDRED_REPRODUCIBLE_DIR + "data/validation/Reddit/" + EMBEDDING_TYPE + "_embedded_features.pckl" # without the certainty level; attached dynamically later
FEATURES_OUT = MEDRED_REPRODUCIBLE_DIR + "data/validation/Reddit/" + EMBEDDING_TYPE + "_all_results.csv"
# where to save models
MODEL_OUT_DIR = MEDRED_REPRODUCIBLE_DIR + "resources/predictors/"



Further preprocessing for xgboost models.

In [5]:
def prepare_training_data_for_one_disease(df, disease):
    '''
    Tags a single disease in the data for classification, labeling each either 1 or 0.
    
    Originally been built with oversampling for feature balancing, but executing portions have this hardcoded to off. As such,
    this feature is removed in favor of retaining simpler and more legible code.

    df: data frame o fpreprocessed features for all diseases
    disease: disease category being predicted; originally named DISEASE7s for unstated reasons--legacy naming from early draft?
    '''
    
    # number of rows corresponding to the disease being predicted
    dis_size = len(df[df['disease']==disease])
    sample_size = dis_size
    
    # denote target disease (positive class)
    df_dis = df[df['disease'] == disease]
    df_dis = df_dis.sample(n=sample_size, random_state=7).reset_index()
    df_dis['disease'] = 1
    
    # denote all other diseases (negative class)
    df_others = df[df['disease'] != disease]
    df_others = df_others.sample(n=sample_size, random_state=7).reset_index()
    df_others['disease'] = 0
    
    # concat negative and positive records back together
    df_sample = pd.concat([df_dis, df_others])
    df_sample = df_sample.drop(columns=['index'])
    
    # format and return
    training = df_sample.copy()
    training = training.reset_index(drop=True)
    return training

In [27]:
def XGBoost_cross_validate(training, disease_number_labels):
    '''
    5-fold CV binary classification with xgboost, trained with 1k trees and max tree depth of 4.
    Configured to use 11 parallel cores via n_jobs in the model fit call.
    The evaluation metric is overriden to "error" to match defautl at time of MedRed publishing.
    '''
    training_labels = training["disease"].astype(int)
    training_features = pd.DataFrame(training["features"].tolist())
    
    AUC_results = []
    f1_results = []
    results = []

    # set up folds
    kf = StratifiedKFold(n_splits=5, random_state=7, shuffle=True)
    # train across folds
    for train_index, test_index in kf.split(training_features,training_labels):
        # split out data
        #   training set
        X_train = training_features.loc[train_index]
        y_train = training_labels.loc[train_index]
        #   test set
        X_test = training_features.loc[test_index]
        y_test = training_labels.loc[test_index]

        # fit model
        model = XGBClassifier(n_estimators=50, n_jobs=11, max_depth=4, eval_metric="error")  # 1000 200
        model.fit(X_train, y_train.values.ravel())
        
        # get test set predictions to use for metrics
        predictions = model.predict(X_test)
        
        # check performance
        results.append(precision_recall_fscore_support(y_test, predictions))
        f1_results.append(f1_score(y_true=y_test, y_pred=predictions, average='weighted'))
        AUC_results.append(metrics.roc_auc_score(y_test, predictions))

    # get test set summary stats across folds
    f1_results_avg = [pd.np.mean(f1_results), pd.np.std(f1_results)]
    AUC_results_avg = [pd.np.mean(AUC_results), pd.np.std(AUC_results)]
    
    return f1_results_avg, AUC_results_avg, results, model

def eval_functions(f1_results_avg, AUC_results_avg, results):
    '''
    Aggregate perfomance metrics across folds.
    '''
    results_avg = pd.np.mean(results, axis=0)
    results_std = pd.np.std(results, axis=0)
    P_res = np.array([np.mean(results_avg[0]), np.mean(results_std[0])])
    R_res = np.array([np.mean(results_avg[1]), np.mean(results_std[1])])
    support_res = np.array([np.mean(results_avg[3]), np.mean(results_std[3])])
    print("F1 average score ", f1_results_avg, np.mean(results_avg[2]))
    print("AUC average score ", AUC_results_avg)
    print("P average score ", P_res)
    print("R average score ", R_res)
    print("Support per class ", support_res)
    
    return {'F1':f1_results_avg, 'AUC': AUC_results_avg, 'support': support_res,
           'P': P_res, 'R': R_res}

Define pipeline from preprocessed features to modeling.

The diseases modeled may controlled by adjusting `all_sr` here.

In [37]:
def predict_with_certainty(certainty, features_file=FEATURES_IN, results_file=FEATURES_OUT):

    # define diseases for which to model
    all_sr = ['bpd', 'cfs','crohnsdisease', 'dementia',  'depression',\
            'diabetes', 'dysautonomia', 'gastroparesis','hypothyroidism', 'ibs', \
            'interstitialcystitis', 'kidneystones', 'menieres', 'multiplesclerosis',\
            'parkinsons', 'psoriasis']
    # all_sr = ['sleepapnea']
    # all_sr = ['rheumatoid']
    disease_values_dict = {el:i for i, el in enumerate(all_sr)}
    disease_names = list(disease_values_dict.keys())
    disease_labels = list(disease_values_dict.values())

    # load features with desired certainty cutoff
    #   get input file name
    features_file = features_file.replace(".pckl", "_{:.2f}.pckl".format(certainty))
    results_file = results_file.replace(".csv", "_{:.2f}.csv".format(certainty))
    #   read in
    features = pd.read_pickle(features_file)

    # minimal final preprocessing
    features.rename(columns={'vec':'features'}, inplace=True)
    features = features.drop(columns=['subreddit', 'entities'])
    disease = features['disease']
    print ("Post count per disease is", {d:n for d, n in zip(all_sr, features.groupby('disease').size())})

    # get the classes sizes for the smallest type of disease in each model (before 1/0 encoding)
    #   can potentially help inform over/under sampling decisions
    # min_class_size = min(features.groupby('disease').size())
    # print("Class sizes", min_class_size)

    print('Distribution before imbalancing: {}'.format(Counter(disease)))

    all_res = defaultdict(int) # for metrics
    # for each disease, train a model
    for disease in disease_labels:
        print(disease)
        # labels to predict
        disease_number_labels = [0, 1]
        # prep the data
        balanced_features = prepare_training_data_for_one_disease(features, disease)
        # run the classifier with CV
        f1_results_avg, AUC_results_avg, results, model = \
            XGBoost_cross_validate(balanced_features, disease_number_labels)
        # show performance metrics
        print("RESULTS for ~~~~~~~~~~~~~~~~ ", disease_names[disease], str(disease), "~~~~~~~~~~~~~~~~~")
        res = eval_functions(f1_results_avg, AUC_results_avg, results)
        all_res[disease_names[disease]] =  res
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        # save model
        pickle.dump(model, open(MODEL_OUT_DIR + disease_names[disease] + "_" + EMBEDDING_TYPE + "_" + str(certainty) + ".pickle.dat", "wb"))

    df_res = pd.DataFrame(all_res)
    df_res.to_csv(results_file)

Run the models. Note that the bug preventing appropriate tagging for rheumatoid arthritis will cause that instance to fail if configured. As-is, it was simply excluded. Additionally, the sleep apnea data tends to have issues with stratification which require it to be run separately.

In [38]:
predict_with_certainty(0.90, features_file=FEATURES_IN, results_file=FEATURES_OUT)

Post count per disease is {'bpd': 14138, 'cfs': 5135, 'crohnsdisease': 13106, 'dementia': 764, 'depression': 54731, 'diabetes': 20168, 'dysautonomia': 916, 'gastroparesis': 375, 'hypothyroidism': 4163, 'ibs': 8522, 'interstitialcystitis': 1002, 'kidneystones': 729, 'menieres': 375, 'multiplesclerosis': 6353, 'parkinsons': 328, 'psoriasis': 2916}
Distribution before imbalancing: Counter({4: 54731, 5: 20168, 0: 14138, 2: 13106, 9: 8522, 13: 6353, 1: 5135, 8: 4163, 15: 2916, 17: 2759, 10: 1002, 6: 916, 3: 764, 11: 729, 12: 375, 7: 375, 14: 328})
0


c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of lab

RESULTS for ~~~~~~~~~~~~~~~~  bpd 0 ~~~~~~~~~~~~~~~~~
F1 average score  [0.8393069710079978, 0.004061015075943679] 0.8393067628769069
AUC average score  [0.8395448591551762, 0.0039946285494548575]
P average score  [0.84155922 0.00734913]
R average score  [0.83954486 0.01028902]
Support per class  [2.82760000e+03 4.89897949e-01]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1


c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of lab

RESULTS for ~~~~~~~~~~~~~~~~  cfs 1 ~~~~~~~~~~~~~~~~~
F1 average score  [0.8246169352810405, 0.009869310106071805] 0.8246169352810405
AUC average score  [0.824732229795521, 0.009790505916455255]
P average score  [0.82554573 0.01430545]
R average score  [0.82473223 0.01966983]
Support per class  [1027.    0.]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2


...

RESULTS for ~~~~~~~~~~~~~~~~  multiplesclerosis 13 ~~~~~~~~~~~~~~~~~
F1 average score  [0.8627022214101195, 0.006182726708452204] 0.8627019280488206
AUC average score  [0.8627404796273007, 0.006166453824929683]
P average score  [0.86315335 0.00910952]
R average score  [0.86274048 0.0114573 ]
Support per class  [1.27060000e+03 4.89897949e-01]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
14


c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of lab

RESULTS for ~~~~~~~~~~~~~~~~  parkinsons 14 ~~~~~~~~~~~~~~~~~
F1 average score  [0.8139134941462671, 0.03446572015638429] 0.8138987146775358
AUC average score  [0.8140326340326339, 0.03434505510771586]
P average score  [0.81493674 0.0361381 ]
R average score  [0.81403263 0.04201988]
Support per class  [65.6         0.48989795]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
15


c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of lab

RESULTS for ~~~~~~~~~~~~~~~~  psoriasis 15 ~~~~~~~~~~~~~~~~~
F1 average score  [0.8969327272764808, 0.010127733252645764] 0.8969324217207393
AUC average score  [0.8969454169505863, 0.01012506869529715]
P average score  [0.89717231 0.01242908]
R average score  [0.89694542 0.01372996]
Support per class  [5.832e+02 4.000e-01]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Now the sleep apnea model.

In [39]:
def predict_with_certainty(certainty, features_file=FEATURES_IN, results_file=FEATURES_OUT):

    # define diseases for which to model
    # all_sr = ['bpd', 'cfs','crohnsdisease', 'dementia',  'depression',\
    #         'diabetes', 'dysautonomia', 'gastroparesis','hypothyroidism', 'ibs', \
    #         'interstitialcystitis', 'kidneystones', 'menieres', 'multiplesclerosis',\
    #         'parkinsons', 'psoriasis']
    all_sr = ['sleepapnea']
    # all_sr = ['rheumatoid']
    disease_values_dict = {el:i for i, el in enumerate(all_sr)}
    disease_names = list(disease_values_dict.keys())
    disease_labels = list(disease_values_dict.values())

    # load features with desired certainty cutoff
    #   get input file name
    features_file = features_file.replace(".pckl", "_{:.2f}.pckl".format(certainty))
    results_file = results_file.replace(".csv", "_{:.2f}.csv".format(certainty))
    #   read in
    features = pd.read_pickle(features_file)

    # minimal final preprocessing
    features.rename(columns={'vec':'features'}, inplace=True)
    features = features.drop(columns=['subreddit', 'entities'])
    disease = features['disease']
    print ("Post count per disease is", {d:n for d, n in zip(all_sr, features.groupby('disease').size())})

    # get the classes sizes for the smallest type of disease in each model (before 1/0 encoding)
    #   can potentially help inform over/under sampling decisions
    # min_class_size = min(features.groupby('disease').size())
    # print("Class sizes", min_class_size)

    print('Distribution before imbalancing: {}'.format(Counter(disease)))

    all_res = defaultdict(int) # for metrics
    # for each disease, train a model
    for disease in disease_labels:
        print(disease)
        # labels to predict
        disease_number_labels = [0, 1]
        # prep the data
        balanced_features = prepare_training_data_for_one_disease(features, disease)
        # run the classifier with CV
        f1_results_avg, AUC_results_avg, results, model = \
            XGBoost_cross_validate(balanced_features, disease_number_labels)
        # show performance metrics
        print("RESULTS for ~~~~~~~~~~~~~~~~ ", disease_names[disease], str(disease), "~~~~~~~~~~~~~~~~~")
        res = eval_functions(f1_results_avg, AUC_results_avg, results)
        all_res[disease_names[disease]] =  res
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        # save model
        pickle.dump(model, open(MODEL_OUT_DIR + disease_names[disease] + "_" + EMBEDDING_TYPE + "_" + str(certainty) + ".pickle.dat", "wb"))

    df_res = pd.DataFrame(all_res)
    df_res.to_csv(results_file)

predict_with_certainty(0.90, features_file=FEATURES_IN, results_file=FEATURES_OUT.replace(".csv", "2.csv"))


Post count per disease is {'sleepapnea': 14138}
Distribution before imbalancing: Counter({4: 54731, 5: 20168, 0: 14138, 2: 13106, 9: 8522, 13: 6353, 1: 5135, 8: 4163, 15: 2916, 17: 2759, 10: 1002, 6: 916, 3: 764, 11: 729, 12: 375, 7: 375, 14: 328})
0


c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\alex\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of lab

RESULTS for ~~~~~~~~~~~~~~~~  sleepapnea 0 ~~~~~~~~~~~~~~~~~
F1 average score  [0.8393069710079978, 0.004061015075943679] 0.8393067628769069
AUC average score  [0.8395448591551762, 0.0039946285494548575]
P average score  [0.84155922 0.00734913]
R average score  [0.83954486 0.01028902]
Support per class  [2.82760000e+03 4.89897949e-01]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Finally, evaluate the correlation between post count and F1 score, and the number of drugs and F1 score--per condition.

In [67]:
# load features with desired certainty cutoff
#   read in
features = pd.read_pickle(FEATURES_IN.replace(".pckl", "_{:.2f}.pckl".format(0.9)))
# define diseases
all_sr = ['bpd', 'cfs','crohnsdisease', 'dementia',  'depression',\
            'diabetes', 'dysautonomia', 'gastroparesis','hypothyroidism', 'ibs', \
            'interstitialcystitis', 'kidneystones', 'menieres', 'multiplesclerosis',\
            'parkinsons', 'psoriasis', 'sleepapnea']
# minimal final preprocessing, as before
features.rename(columns={'vec':'features'}, inplace=True)
features = features.drop(columns=['subreddit', 'entities'])
disease = features['disease']
# get post counts per disease
post_count = pd.DataFrame([(d, n) for d, n in zip(all_sr, features.groupby('disease').size())], columns=['disease', 'n_posts'])
post_count = post_count.set_index('disease')
# get F1 scores
f1 = pd.read_csv(FEATURES_OUT.replace(".csv", "_{:.2f}.csv".format(0.9)))
post_count = post_count.join(f1.iloc[0])
post_count.columns = ["n_posts", "F1"]
# add sleep apnea, since run separately
f1_sleepapnea = pd.read_csv(FEATURES_OUT.replace(".csv", "2_{:.2f}.csv".format(0.9))).iloc[0, 1]
post_count.loc["sleepapnea", "F1"] = f1_sleepapnea
# extract mean F1 score for each disease
post_count["F1"] = [ast.literal_eval(i)[0] for i in post_count["F1"]]
# get correlation
print(post_count)
print(np.corrcoef(post_count.n_posts, post_count.F1))

,n_posts,F1
disease,,
bpd,14138,0.839307
cfs,5135,0.824617
crohnsdisease,13106,0.852477
dementia,764,0.850935
depression,54731,0.884715
diabetes,20168,0.908237
dysautonomia,916,0.848145
gastroparesis,375,0.819468
hypothyroidism,4163,0.861201
